# Embeddings Neuronales


**TODO**

### Importanción de librería requeridas

In [1]:
import gensim.corpora as corpora
from gensim.models import Word2Vec

from collections import Counter
import numpy as np

import pickle
import pandas as pd

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline


### Definición de variables globales

In [2]:
TEXT_FILE_READ = 'docs/preprocessing_reddit_data.csv'
TEXT_SAVE_FILE = 'docs/reddit_data_lda.csv'
FILENAME_PICKLE = "docs/tmpreddit.pickle"
n_clusters = 70

### Lectura de los comentarios de Reddit

Los comentarios fueron previamente preprocesados (Ver en TODO).

In [3]:
with open(FILENAME_PICKLE, 'rb') as f:
    df = pickle.load(f)


### Vocabulario

In [4]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemma_tokens'])

# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)

# Creating a corpus object
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]


In [5]:
processed_corpus = df['lemma_tokens']


### Entrenamiento del modelo Word2Vec

In [6]:
model = Word2Vec(sentences=processed_corpus, vector_size=100, window=5, min_count=1, workers=4)
model.train(processed_corpus, total_examples=len(processed_corpus), epochs=100)
model.save("word2vec.model")

In [7]:
word_vecs = []
vocabulary = list(model.wv.key_to_index)

for key in model.wv.key_to_index:
    word_vecs.append(model.wv[key])

In [8]:
# algunas predicciones

model.wv.most_similar("rucula")

[('jajajajjajaajaj', 0.9567767977714539),
 ('laconcho', 0.9120215177536011),
 ('bottle', 0.9049957394599915),
 ('branding', 0.8658552765846252),
 ('submarinar', 0.7980161905288696),
 ('golden', 0.7718115448951721),
 ('meanies', 0.7614740133285522),
 ('wormtongue', 0.6600823998451233),
 ('eyes', 0.6527519226074219),
 ('aragorn', 0.6500346064567566)]

### Generación de vectores desde documentos

In [9]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(processed_corpus, model=model)
len(vectorized_docs), len(vectorized_docs[0])



(27791, 100)

### Generación de clusters

In [10]:
def mbkmeans_clusters(
	X, 
    k, 
    mb, 
    print_silhouette_values, 
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [11]:
clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_docs,
    k=n_clusters,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": df["body"].values,
    "tokens": [" ".join(text) for text in processed_corpus],
    "cluster": cluster_labels
})



For n_clusters = 70
Silhouette coefficient: 0.01
Inertia:956541.2375425596


KeyboardInterrupt: 

### *Top terms* por cluster (basado en los centroides de los clusters)

In [12]:
print("Most representative terms per cluster (based on centroids):")
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")



Most representative terms per cluster (based on centroids):
Cluster 0: amigo autorizacion jajsjsjj empija happn 
Cluster 1: andar bici sociales beiby zorzal 
Cluster 2: va catre enojarte desmechado amigoooo 
Cluster 3: vivir burbuja unicenter modestamente veiir 
Cluster 4: ⡾ él ⠑ ⣇ ⠣ 
Cluster 5: pagar impuesto ganancia jejar cobrar 
Cluster 6: tomar mundo baso bombillar ⠣ 
Cluster 7: precio regatear listado inflacion elefant 
Cluster 8: ⡾ ⠑ jsjajar cagar tiento 
Cluster 9: ver video ⡾ guille aquino 
Cluster 10: mujer hombre desabrochar testosterona dalbon 
Cluster 11: dar sudar ⠑ ⡾ robartir 
Cluster 12: votar peronismo oficialismo voto perder 
Cluster 13: tipo capaz dirijar pelotudo ⡾ 
Cluster 14: sexual depredador degenerados malo taaaan 
Cluster 15: ⡾ atardecer avenger iluminano tortuoso 
Cluster 16: ⡾ ⠑ ballotage aniquilar atribuis 
Cluster 17: reboton dudamente noroeste 17695 escobas 
Cluster 18: vo seguí espontaneo hegemonía decís 
Cluster 19: grin drop exciting ape performing 
Cl

### *Top terms* por cluster (basado en las palabras más frecuentes)

In [13]:
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_frequent = Counter(" ".join(df_clusters.query(f"cluster == {i}")["tokens"]).split()).most_common(5)
    for t in most_frequent:
        tokens_per_cluster += f"{t[0]}({str(t[1])}) "
    print(f"Cluster {i}: {tokens_per_cluster}")


Cluster 0: amigo(161) pasar(4) tenés(4) seguro(4) terminar(4) 
Cluster 1: andar(123) llamar(33) bici(18) él(9) ver(7) 
Cluster 2: va(205) pod(29) orto(16) él(13) vo(12) 
Cluster 3: vivir(203) pobre(18) gente(16) él(12) país(11) 
Cluster 4: él(585) seguir(113) pasar(78) yo(73) vender(67) 
Cluster 5: pagar(224) impuesto(125) cobrar(53) sueldo(31) él(28) 
Cluster 6: tomar(183) mundo(68) mate(24) él(16) decisión(12) 
Cluster 7: precio(219) inflación(40) dólar(36) vender(32) bajar(31) 
Cluster 8: cagar(132) matar(74) risa(68) morir(61) él(45) 
Cluster 9: ver(378) video(105) él(50) cara(48) mirar(31) 
Cluster 10: mujer(124) hombre(112) gris(9) él(8) trans(8) 
Cluster 11: dar(241) él(89) vuelta(15) querer(12) yo(10) 
Cluster 12: peronismo(133) votar(131) voto(89) perder(89) elección(32) 
Cluster 13: tipo(280) pelotudo(17) él(14) persona(13) pobre(10) 
Cluster 14: malo(63) sexual(19) so(11) arma(9) educación(8) 
Cluster 15: ver(63) parecer(45) serie(44) él(38) película(32) 
Cluster 16: polític

### Recupere los documentos más representativos (basados en los centroides de los clústeres) para un cluster en particular

In [14]:
test_cluster = 20
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:10]:
    print( df["body"].values[d])
    print("-------------")

El tema es que no soy comerciante. Nisiquiera tengo cuit.
-------------
Eso es otro tema que engloba otras tácticas complejas de exponer.
-------------
Ciro Pertusi tiene un historial MUY largo con tema pedofilia, no es solo esa letra.
-------------
Bueno pero como abordas temas como el aborto adonde necesariamente se prohíbe desde el estado?
-------------
No es un tema prioritario pero quiere hacer un plebiscito?
-------------
"... hay publicaciones científicas del tema".. Fuente: Arial 12
-------------
Hay que apurar el tema de eutanasia.
-------------
Que temas escuchaste? Yo lo conoci por Bancate ese defecto, y me volo la cabeza ese tema
-------------
Por ejemplo, la parte de promoción/publicidad es un quilombo y el tema de packaging es otra cagada. Perdóname pero me parece muy limitado el trabajo que se hizo para ser algo que tiene años.
-------------
Más o menos, Les Luthiers no era stand up. Había mucho tema de música y el humor era, en su mayoría, lingüístico
-------------


In [15]:

#print(len(vectorized_docs))
#print(vectorized_docs[0])

test_v = vectorize([['defender', 'peso', 'siente', 'corazón', 'compro', 'pesos', 'tasa', 'fijo', 'año']], model=model)
prediction = clustering.predict(test_v)
print(prediction)

[62]


In [38]:
reddit = pd.read_csv(TEXT_FILE_READ)

def get_cluster(row):
    test_v = vectorize([row], model=model)
    return clustering.predict(test_v)

reddit['cluster'] = reddit.apply(lambda row: get_cluster(row['lemma_tokens']) , axis = 1) 


In [39]:
# Show
reddit.head(10)

,score,id,flair,comms_num,body,comment_parent_id,is_replay,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,lemma_tokens,body_preprocessing,cluster
0,1,hfw14mt,Discusion🧐,1,todo para decir que tapaste el baño. tira un b...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tapastir', 'baño', 'tirar', 'balde', 'aguo']",tapastir baño tirar balde aguo,[67]
1,1,hfw41eh,Discusion🧐,0,"sopapa primero master, si hay tapón te vas a t...",hfw14mt,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapa', 'master', 'tapón', 'va', 'teñir', '...",sopapa master tapón va teñir medio,[67]
2,1,hfw1ao2,Discusion🧐,0,"Usas la sopapa, o tiras agua caliente con un b...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapo', 'tira', 'agua', 'caliente', 'balde']",sopapo tira agua caliente balde,[67]
3,1,hfw3jof,Discusion🧐,2,Lo que he probado que siempre me dio resultado...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['probado', 'resultado', 'sellar', 'boca', 'in...",probado resultado sellar boca inodoro tirar ca...,[67]
4,1,hfw6v4i,Discusion🧐,0,Estas cobrando por dar mantenimiento y no sabe...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['cobrar', 'mantenimiento', 'carajo', 'kjjjjjj...",cobrar mantenimiento carajo kjjjjjjjjj vivirio...,[67]
5,1,hfw26iv,Discusion🧐,0,"Si tenes algo con punta, metelo y hace un poco...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tén', 'punto', 'metelo', 'fuerza', 'romper',...",tén punto metelo fuerza romper tapo baño tirar...,[67]
6,1,hfw2gof,Discusion🧐,1,"Con una manguera para regar el jardín, si tene...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['regar', 'jardín', 'tén', 'pod', 'probar']",regar jardín tén pod probar,[67]
7,1,hfw5s13,Discusion🧐,0,"despues regas el jardin y se lava sola, solo q...",hfw2gof,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['rega', 'jardin', 'lava', 'tenés', 'lavarte',...",rega jardin lava tenés lavarte mano pulgar chorro,[67]
8,1,hfw3air,Discusion🧐,0,La respuesta real es que se venden unos caños ...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['respuesta', 'real', 'vender', 'caño', 'alamb...",respuesta real vender caño alambrado decir cañ...,[67]
9,7,hfvxa6w,Discusion🧐,3,Mi alfajor favorito es el Havana,q443eo,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['alfajor', 'favorito', 'haván']",alfajor favorito haván,[67]


In [40]:
reddit.to_csv(TEXT_SAVE_FILE, index=False)

In [42]:
for i in range(n_clusters):
    reddit[(reddit["cluster"] == i)][['flair', 'body']].to_csv('docs/testlda/' + str(i) + '.csv')
